### RFC

### import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
import matplotlib.pyplot
import pickle

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)


from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE, ADASYN

import xgboost as xgb

%matplotlib inline

### Load variables stored by data_preproccessing notebook

In [2]:
%store -r train_data_formodel
%store -r test_data
%store -r my_data
%store -r uniques
%store -r best_feats



### configurations
* save_plots -> True|False
* random_seed_state -> number, sets random state for model and for stratified splits 
* classify_bedrock_only -> True|False
* pickle_model -> True|False, wether model should be serialised and saved
* pickle_model_name -> string, name of serialised model
* grid_search -> True|False, if set to true then grid search is performed to identify optimum hyperparamaters for model 
* scale -> True|False if set to True then features scaled to all have mean value 0 and standard deviation 1
* pickle_file_path -> string,  filepath for serialised model to be saved to

In [3]:
save_plots = False
random_seed_state = 42
classify_bedrock_only = False
pickle_model = False
pickle_model_name = 'grouped'
pickle_file_path = '../../../model'
grid_search = False
scale = False

### if only bedrock sites are classified then classes are label encoded, if bedrock sites alone are not being classified then the class sites would have already been label encoded in the 1 data_preproccessing notebook 

In [4]:
if classify_bedrock_only:
    train_data_formodel['class'], uniques = pd.factorize(train_data_formodel['class'])
    train_data_formodel = train_data_formodel[train_data_formodel['Geology']=='Bedrock']

### counts of instances in all classes before oversampling

In [5]:
train_data_formodel['class'].value_counts()

22    120
4     105
23    105
16    100
21     74
17     61
24     60
0      53
12     45
14     36
2      36
15     36
6      30
11     30
10     30
7      30
20     28
5      27
8      27
19     27
1      24
13     21
3      18
18     18
9      17
Name: class, dtype: int64

### The class column is stored as the variable y 

In [6]:
y = np.array(train_data_formodel['class'])

### The variables identified as best by the 2 feature_selection notebook are used as features

In [7]:
train_data_feats = train_data_formodel[best_feats]

### address class imbalance using synthetic minority oversampling technique (SMOTE) algorithm

In [8]:
if scale:
    my_scaler = StandardScaler()
    X = np.array(my_scaler.fit_transform(np.array(train_data_feats)))
else:
    X = np.array(np.array(train_data_feats))

### the dimensions of the class and features are checked

In [10]:
print(X.shape)
print(y.shape)

(3000, 31)
(3000,)


### Grid search is done to identify best hyperparamaters for the model

In [13]:
%%time
if grid_search:
    esti = xgb.XGBClassifier(booster = 'gbtree')


    
    n_estimators = [int(x) for x in np.linspace(start = 4000, stop = 8000, num = 4)]
    max_depth = [int(x) for x in np.linspace(10, 60, num = 3)]
    min_child_weight = [0,3]
    gamma = [0,3]

    
    
    random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
                   'min_child_weight':min_child_weight,
                   'gamma':gamma
                  }
    
    clf = RandomizedSearchCV(estimator = esti, n_iter=20, param_distributions = random_grid,
                              n_jobs=-1, scoring='f1_macro', cv = 3, verbose=3, 
                              random_state = random_seed_state)
    
    clf.fit(X, y)
    print(clf.best_params_)


CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.68 µs


best paramaters  from the randomised grid search were {'n_estimators': 6666, 'min_child_weight': 3, 'max_depth': 10, 'gamma': 0}, took 149 mins to fit 60 models

In [16]:
if grid_search:
    esti = xgb.XGBClassifier(booster = 'gbtree')

    n_estimators = [5000, 6000, 7000]
    min_child_weight = [3, 5]
    max_depth = [5,10]

    
    
    random_grid = {'n_estimators': n_estimators,
               'min_child_weight': min_child_weight,
                  'max_depth':max_depth
                  }
    
    clf = GridSearchCV(estimator = esti, param_grid = random_grid,
                              n_jobs=-1, scoring='f1_macro', cv = 3, verbose=3 
                              )
    clf.fit(X, y)
    print(clf.best_params_)


best paramaters from the grid search were {'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 5000}


### Carry out 10-f0ld stratified cross validation, class f1 scores and macro f1 scores with weighted averages are calculated

In [17]:
esti = xgb.XGBClassifier(booster="gbtree", max_depth = 5, min_child_weight=3, n_estimators=5000)
skf = StratifiedKFold(n_splits=10, random_state=random_seed_state)
skf.get_n_splits(X, y)
class_f1_scores = []
macro_f1_scores = []
accuracy_scores = []
feat_imp =[]
f1_dict = {}
feat_imp_dict = {}
count = 0
for train_index, test_index in skf.split(X, y):
    count = count + 1
    print('making model:')
    key = 'round' + str(count)
    print(count)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_post_smote, y_post_smote = SMOTE(random_state=42).fit_sample(X_train, y_train)
        
    esti.fit(X_train, y_train)
    y_pred = esti.predict(X_test)
    class_f1_scores = f1_score(y_test, y_pred, average = None)
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    macro_f1_scores.append(f1_score(y_test, y_pred, average = 'weighted'))
    f1_dict[key] = class_f1_scores 
    feat_imp_dict[key] = esti.feature_importances_

making model:
1


/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


making model:
2


/home/rmorse/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


making model:
3


KeyboardInterrupt: 

In [ ]:
f1_df = pd.DataFrame(data = f1_dict)


In [ ]:
for key in f1_dict:
    print(len(f1_dict[key]))

### Below are the encodings for the class variable

In [ ]:
print(train_data_formodel['class'].unique())
print(list(uniques))

In [ ]:
f1_df_final = pd.concat([f1_df, pd.Series(uniques)], axis = 1)

In [ ]:
f1_df_final.rename(columns={0:'class'}, inplace=True)
f1_df_final.set_index('class', drop = True, inplace = True)

### Boxplot showing the distribution of class f1 scores from 10 models

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
plot = sns.boxplot(data = f1_df_final.T)
plot.set_title('F1 scores for each site', fontdict={'fontsize': 14})
plot.set_ylabel('F1 score', fontdict={'fontsize': 11})
plot.set_xlabel("Bedrock site or superficial site", fontdict={'fontsize': 11})

if save_plots == True:
    fig = plot.get_figure()
    fig.savefig('output/site_specific_f1_scores.png')

In [ ]:
#pd.DataFrame(data = f1_df_final.T.median()).to_csv('output/median_class_f1_scores.csv')

### Boxplot showing the macro F1 score with weighted averages

In [ ]:
plot = sns.boxplot(macro_f1_scores)
plot.set_title('Average-weighted macro-f1 score', fontdict={'fontsize': 14})
plot.set_xlabel("F1-score", fontdict={'fontsize': 11})

if save_plots == True:
    fig = plot.get_figure()
    fig.savefig('output/macro_f1_scores.png')

In [ ]:
#pd.Series(pd.Series(macro_f1_scores).median()).to_csv('output/median_macro_f1.csv')

In [ ]:
pd.Series(macro_f1_scores).median()

### Boxplot showing accuracy scores

In [ ]:
sns.boxplot(accuracy_scores)

### Get feature importances

feat_imp_df = pd.DataFrame(data = feat_imp_dict)
feat_imp_df.head()

feat_imp_df_final = pd.concat([feat_imp_df, pd.Series(my_data[my_data.columns.values[9:-1]].columns.values)], axis = 1)
feat_imp_df_final.rename(columns = {0:'element'}, inplace = True )
feat_imp_df_final.head()

feat_imp_df_final.set_index('element', inplace=True)


feat_imp_df_final_plot = feat_imp_df_final.T

feat_imp_df_final_plot

elements = feat_imp_df_final_plot.columns.values 
mean_feature_importance = []
for col in list(feat_imp_df_final_plot.columns.values):
    mean_feature_importance.append(feat_imp_df_final_plot[col].mean())
    

mean_feature_importance_df = pd.concat([pd.Series(elements), pd.Series(mean_feature_importance)], axis = 1)

mean_feature_importance_df.rename(columns={0:'elements', 1:'mean_importance'}, inplace=True)

mean_feature_importance_df.sort_values(by='mean_importance', ascending=False, inplace=True)

ordered_col_names = list(mean_feature_importance_df['elements'])

sns.set_style("whitegrid")
sns.set_style()
sns.set(rc={'figure.figsize':(20,20)})
plot = sns.boxplot(data = feat_imp_df_final_plot[ordered_col_names])
plot.set_xticklabels(plot.get_xticklabels(),rotation=90, ha = 'left')
plot.set_title('Feature (element) importance', fontdict={'fontsize': 20})
plot.set_ylabel('Feature importance', fontdict={'fontsize': 15})
plot.set_xlabel("Element", fontdict={'fontsize': 15})

if save_plots == True:
    fig = plot.get_figure()
    fig.savefig('output/feature_importances.png')

### Model is built for predicting source of artefacts 

esti_final = xgb.XGBClassifier()

esti_final.fit(X, y)

In [ ]:
if pickle_model == True:
    pickle.dump(esti_final, open(pickle_file_path+'.sav', 'wb'))

print(train_data_formodel['class'].unique())
print(uniques)

df_for_identifiers = test_data.copy(deep = True)
identifiers =  df_for_identifiers['Analysis']

### Predictions are made for the artefacts

y_pred = esti_final.predict(np.array(test_data[test_data.columns.values[9:-1]]))

y_pred_proba = esti_final.predict_proba(np.array(test_data[test_data.columns.values[9:-1]]))


probabilities_df = pd.DataFrame(data = y_pred_proba, columns = uniques)
probabilities_df_final = pd.concat([probabilities_df, pd.Series(list(identifiers))], axis = 1)

probabilities_df_final.rename(columns = {0:'identifier'}, inplace=True)

final_pred_df = pd.concat([pd.Series(y_pred), probabilities_df_final], axis = 1)

final_pred_df.rename(columns={0:'class_number'}, inplace = True)

### predictions are outputted as csv file

final_pred_df.to_csv('output/predictions.csv')


uniques_list = list(uniques)
def get_pred_names(row):
    return(uniques_list[row['class_number']])
final_pred_df['class_predictions'] = final_pred_df.apply(get_pred_names, axis = 1)

final_pred_df.head()